In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, ConvLSTM2D, BatchNormalization, Conv3D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# ------------------------
# Chargement des données prétraitées
# ------------------------
dossier = r"C:\Users\Ichrak\Desktop\Mission entreprise"
images_ped1 = np.load(os.path.join(dossier, "UCSDped1_train.npy"))
images_ped2 = np.load(os.path.join(dossier, "UCSDped2_train.npy"))

print(f"UCSDped1: {images_ped1.shape}, UCSDped2: {images_ped2.shape}")

# ------------------------
# Création des séquences
# ------------------------
def creer_sequences(images, seq_len=10):
    return np.array([images[i:i+seq_len] for i in range(len(images) - seq_len + 1)], dtype=np.float32)

sequence_length = 10
sequences_ped1 = creer_sequences(images_ped1, sequence_length)
sequences_ped2 = creer_sequences(images_ped2, sequence_length)

# Fusionner les deux bases
sequences = np.concatenate((sequences_ped1, sequences_ped2), axis=0)
sequences = sequences[..., np.newaxis]  # Ajouter canal (grayscale)
print(f"Forme finale des séquences : {sequences.shape}")  # (nb_seq, 10, 64, 64, 1)

# ------------------------
# Modèle ConvLSTM AutoEncoder
# ------------------------
input_layer = Input(shape=(sequence_length, 64, 64, 1))
x = ConvLSTM2D(filters=64, kernel_size=(3,3), padding='same', return_sequences=True, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = ConvLSTM2D(filters=32, kernel_size=(3,3), padding='same', return_sequences=True, activation='relu')(x)
x = BatchNormalization()(x)
x = ConvLSTM2D(filters=64, kernel_size=(3,3), padding='same', return_sequences=True, activation='relu')(x)
x = BatchNormalization()(x)
output_layer = Conv3D(filters=1, kernel_size=(3,3,3), activation='sigmoid', padding='same')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mse')
model.summary()

# ------------------------
# Entraînement
# ------------------------
x_train, x_val = train_test_split(sequences, test_size=0.2, random_state=42)
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    x_train, x_train,
    validation_data=(x_val, x_val),
    epochs=20,
    batch_size=8,
    callbacks=[early_stop]
)

# ------------------------
# Visualisation des pertes
# ------------------------
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Perte entraînement')
plt.plot(history.history['val_loss'], label='Perte validation')
plt.title("Courbe de perte ConvLSTM")
plt.xlabel("Époque")
plt.ylabel("MSE")
plt.legend()
plt.grid(True)
plt.show()


UCSDped1: (6800, 64, 64), UCSDped2: (2550, 64, 64)
Forme finale des séquences : (9332, 10, 64, 64, 1)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10, 64, 64, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d (ConvLSTM2D)        │ (None, 10, 64, 64, 64) │       150,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 10, 64, 64, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 10, 64, 64, 32) │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 10, 64, 64, 32) │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, 10, 64, 64, 64) │       221,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 10, 64, 64, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 10, 64, 64, 1)  │         1,729 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 484,545 (1.85 MB)

 Trainable params: 484,225 (1.85 MB)

 Non-trainable params: 320 (1.25 KB)

Epoch 1/20
